# 0. Instalación

In [1]:
%pip install scikit-learn pandas xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install imblearn SMOTE

Note: you may need to restart the kernel to use updated packages.


# 1. Modelos de Árboles de decisión

**Descripción**

Random Forest es un modelo de aprendizaje supervisado basado en árboles de decisión que emplea el principio de ensamble para mejorar la precisión y la robustez de las predicciones. Este enfoque se utiliza principalmente en datos tabulares y es una de las técnicas más populares y eficientes en tareas de clasificación y regresión.

Además, este modelo pertenece a la categoría de modelos de tipo bagging ya que emplea una combinación de múltiples árboles de decisión entrenados de manera independiente y al final se elige la solución mayoritaria o el promedio de las predicciones. Por lo que Random Forest al permitir emplear múltiples árboles de decisión entrenados de distinta manera, permitiendo reducir el sobreajuste y mejorar la generalización.

**Implementación**

En este proyecto vamos a emplear el primer preprocesado realizado. A continuación para balancear los datos, después de haber realizado un análisis con SMOTE y una técnica de submuestreo para balancear el conjunto de los datos. Dinalmente, se ha llegado a la conlcusión de no emplear ninguna estas técnicas, ya que empeoraban las métricas, por lo que nos hemos decantado en unicamente entrenar los modelos con los datos preprocesados. Se ha dejado un apartado con los resultados obtenidos con SMOTE, el submuestreo y además el empleo del Grid Search Vector, que nos empeoraba también las métricas.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')
#data = pd.read_csv('../../../../data/processed/train_limpieza_v1.csv')
data.head(5)


,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_affiliation_uni,party_affiliation_category_map,...,pos_info_without_stopwords,pos_freq_without_stopwords,lemma_freq_without_stopwords,tag_freq_without_stopwords,processed_subject,speaker_entities,speaker_type,speaker_job_tokens,state_info_tokens,party_affiliation_tokens
0,81f884c64a7,1,china is in the south china sea and (building)...,"china,foreign-policy,military",donald-trump,president-elect,new york,republican,republican,political-affiliation,...,"[{'lemma': 'china', 'pos': 'PROPN', 'tag': 'NN...","Counter({'PROPN': 4, 'NOUN': 4, 'ADJ': 1, 'VER...","Counter({'china': 2, 'south': 1, 'sea': 1, 'bu...","Counter({'NNP': 4, 'NN': 3, 'JJ': 1, 'NNS': 1,...","['china', 'foreign-policy', 'military']",['donald trump'],['PERSON'],"['president', '-', 'elect']","['new', 'york']",['republican']
1,30c2723a188,0,with the resources it takes to execute just ov...,health-care,chris-dodd,u.s. senator,connecticut,democrat,democrat,political-affiliation,...,"[{'lemma': 'resource', 'pos': 'NOUN', 'tag': '...","Counter({'NOUN': 7, 'VERB': 4, 'PROPN': 2, 'AD...","Counter({'resource': 1, 'take': 1, 'execute': ...","Counter({'NN': 4, 'NNS': 3, 'VB': 2, 'NNP': 2,...",['health-care'],['chris dodd'],['PERSON'],"['u.s', '.', 'senator']",['connecticut'],['democrat']
2,6936b216e5d,0,the (wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,political commentator,"washington, d.c.",democrat,democrat,political-affiliation,...,"[{'lemma': 'wisconsin', 'pos': 'PROPN', 'tag':...","Counter({'NOUN': 4, 'PROPN': 1, 'VERB': 1})","Counter({'wisconsin': 1, 'governor': 1, 'propo...","Counter({'NN': 2, 'NNS': 2, 'NNP': 1, 'VBN': 1})","['corporations', 'pundits', 'taxes', 'abc-news...",['donna brazile'],['PERSON'],"['political', 'commentator']","['washington', ',', 'd.c', '.']",['democrat']
3,b5cd9195738,1,says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,non-define,non-define,none,none,other-political-groups,...,"[{'lemma': 'say', 'pos': 'VERB', 'tag': 'VBZ',...","Counter({'NOUN': 9, 'VERB': 1, 'ADJ': 1})","Counter({'say': 1, 'representation': 1, 'ex': ...","Counter({'NN': 8, 'VBZ': 1, 'NNS': 1, 'JJ': 1})","['candidates-biography', 'children', 'ethics',...",['rebecca bradley'],['PERSON'],"['non', '-', 'define']","['non', '-', 'define']",['none']
4,84f8dac7737,0,at protests in wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,non-define,wisconsin,republican,republican,political-affiliation,...,"[{'lemma': 'protest', 'pos': 'NOUN', 'tag': 'N...","Counter({'NOUN': 7, 'VERB': 4, 'ADJ': 3})","Counter({'protest': 1, 'wisconsin': 1, 'propos...","Counter({'NNS': 4, 'NN': 3, 'JJ': 3, 'VBN': 2,...","['health-care', 'labor', 'state-budget']","['republican party', 'wisconsin']","['ORG', 'GPE']","['non', '-', 'define']",['wisconsin'],['republican']


Con esos datos preprocesados, seleccionamos que columnas no van a participar en el proceso de entrenamieto. Se ha decidido no usar esas columnas, ya que de todas las características analizadas, eran las que menos información podían aportar a la hora de decidir si estamos ante una *fake news* o no.

En una primera idea se realizó el entrenamiento con todas las columnas, pese al mal funcionamiento de los modelos de árboles de decisión, se cambió la estrategiay se emplea esta estrategia de seleccionar solo las columnas que aportan información importante y no es redundadnte.

Además, se aplicó la vectorización TF-IDF, permitiendo una representación númerica desde texto, limitando a 5000 las palabras más relevantes del inglés y eliminando palabras que no aportan información relevante. De esta forma se captura la importancia de cada término. 

In [5]:
# Seleccionar la noticia y la etiqueta de veracidad de si es fake new
X = data['statement']  
y = data['label']  

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización de texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

**Configuración de hiperparámetros de Random Forest y entrenamiento**

Para llevar a cabo la configuración de los hiperparámetros hemos tenido en cuenta lo la documentación del modelo. Para el criterio de división empleado, se utiliza Gini ya que proporciona una solución rápida y eficiente en la construcción de los árboles. Además, en cuanto al número de estimadores y la profundidad máxima, es decir, como de grande va a ser nuestro bosque y como de profundo va a ser cada árbol se ha optado por valores más altos. El número de estimadores es elevado, ya que como cada árbol de forma individual tiene que dar su valoración, si se poseen gran cantidad de árboles, el error individual promedio disminuye, y por lo tanto, es más robusto pero tiene mayor consumo computacional.

In [6]:

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        
    class_weight=class_weights, 
    random_state=42
)
rf_model.fit(X_train_tfidf, y_train)

# Realizar predicciones
y_pred = rf_model.predict(X_test_tfidf)

**Evaluación**

In [7]:
# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")

              precision    recall  f1-score   support

           0       0.41      0.67      0.51       923
           1       0.74      0.50      0.60      1762

    accuracy                           0.56      2685
   macro avg       0.58      0.59      0.56      2685
weighted avg       0.63      0.56      0.57      2685

Macro Average F1-Score: 0.5557


Finalmente, guardamos el vectorizador tfidf, empleando la biblioteca joblib. De esta manera, al guardarlo, nos permite poder reutilizarlos posteriamente, ya sea para un reentreno o para poder realizar la submissión de kaggle con los datos para probar.

In [8]:
import joblib
# Guardar el vectorizador TF-IDF
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')


['tfidf_vectorizer.joblib']

A cotninuación se proporcionan 3 planteamientos diferentes
1. Utilizando columnas específicas
2.  Aplicando smote
3. Submuestreo
con los cuales no se han obtenido resultados favorables, por no se entra en profundidad.

### 1.1.2 Random forest solo con columnas determinadas

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Cargar el dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar características relevantes
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']]
y = data['label']  

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto mediante TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# Entrenamiento del modelo 
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        
    class_weight=class_weights, 
    random_state=42
)
rf_model.fit(X_train_tfidf, y_train)

# Predicciones para evaluar el rendimiento del modelo
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.41      0.67      0.51       923
           1       0.74      0.50      0.60      1762

    accuracy                           0.56      2685
   macro avg       0.58      0.59      0.56      2685
weighted avg       0.63      0.56      0.57      2685

Macro Average F1-Score: 0.5557


sin usar statement con one hot encoder

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

# Cargar el dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características relevantes (sin la columna 'statement') y la etiqueta
X = data[['subject', 'speaker', 'speaker_job', 'state_info', 'party_affiliation', 'party_affiliation_uni']]  # Columnas relevantes
y = data['label']  # Etiqueta de veracidad

# Realizar el OneHotEncoder para las columnas categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicar One-Hot Encoding a las columnas categóricas
X_encoded = encoder.fit_transform(X)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',  # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights,  # 'balanced'
    random_state=42
)
rf_model.fit(X_train, y_train)

# Realizar predicciones
y_pred = rf_model.predict(X_test)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.39      0.65      0.49       923
           1       0.72      0.48      0.57      1762

    accuracy                           0.54      2685
   macro avg       0.56      0.56      0.53      2685
weighted avg       0.61      0.54      0.54      2685

Macro Average F1-Score: 0.5318


### 1.1.3 Aplico Smote

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from imblearn.over_sampling import SMOTE  

# Cargar del dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características 
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']] 
y = data['label']  

# Dividimos los datos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto con TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# Aplicamos SMOTE ÚNICAMENTE AL CONJUNTO DE ENTRENAMIENTO!!! (al de prueba no)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

# Entrenamiento del modelo 
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced'
    random_state=42
)
rf_model.fit(X_train_smote, y_train_smote)

# Predicciones
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.39      0.80      0.52       923
           1       0.76      0.33      0.46      1762

    accuracy                           0.49      2685
   macro avg       0.57      0.57      0.49      2685
weighted avg       0.63      0.49      0.48      2685

Macro Average F1-Score: 0.4915


### 1.1.4 Aplico la técnica del submuestreo para el balanceo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Carga del dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características 
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']] 
y = data['label'] 

# Dividimos los datos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# Submuestreo del conjunto de entrenamiento siendo la clase 0 la minotitaria y la clase 1 la mayoritaria
# Vamos a realizar un submuestreo aleatorio igualando el número de ejemplos de ambas clases
df_train = pd.DataFrame(X_train_tfidf.toarray())  
df_train['label'] = y_train
df_train_class_0 = df_train[df_train['label'] == 0]
df_train_class_1 = df_train[df_train['label'] == 1]
n_minority = len(df_train_class_0)
df_train_class_1_under = df_train_class_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases 
df_train_balanced = pd.concat([df_train_class_0, df_train_class_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# Separar las características y etiquetas después del submuestreo
X_train_balanced = df_train_balanced.drop(columns=['label'])
y_train_balanced = df_train_balanced['label']

# Entrenamiento
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced'
    random_state=42
)
rf_model.fit(X_train_balanced, y_train_balanced)

# Predicciones
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.34      0.93      0.50       923
           1       0.67      0.07      0.13      1762

    accuracy                           0.37      2685
   macro avg       0.51      0.50      0.32      2685
weighted avg       0.56      0.37      0.26      2685

Macro Average F1-Score: 0.3178


## 1.2 XGBoost

**Descripción**

Este modelo también se trata de un algoritmo de aprendizaje supervisado basado en el enfoque de boosting, empleando tamién para las tareas de clsificación y regresión. A diferencia del Random Forest, XGBoost, se encarga de construir los árboles de manera secuencial, y cada árbol va a intentar corregir los errores producidos por el árbol precedente.

**Implementación**

La implementación es muy similar a la utilizada en el Random Forest, a diferencia de la configuracion de los hiperparámetros.

Para la configuración, en el caso de XGBoost, el número de estimadores y la profundidad máxima de los árboles se configuraron de manera similar a los valores de Random Forest, siguiendo las recomendaciones de la documentación de XGBoost. Además de estos hiperparámetros, se utilizó la métrica logloss para la evaluación. También se ha configurado la tasa de aprendizaje del modelo es baja, de tal manera que va a permitir que vaya aprendiendo de forma gradual y sea más generalizable, evitando tender al sobreajuste, y en cada árbol al usar datos con tantas características, se va a emplear un 60% de estas distribuidas de forma aleatoria.
Además, otra cosa a destacar es que vamos a codificar las variables categóricas para que se pueda tratar sin problemas.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Carga del dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las columnas relevantes 
X = data[['subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']] 
y = data['label']  

# División de los datos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Codificación de las variables categóricas
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Nos asrguramos que ambas columnas son del mismo tamaño
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1)

# Vectorización del texto mediante TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(data['statement'][X_train.index])  
X_test_tfidf = tfidf_vectorizer.transform(data['statement'][X_test.index]) 

# Concatenamos las características de texto mediante TF-IDF y las características categóricas
X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_encoded.values])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_encoded.values])

# Entrenamos el modelo
sample_weights = y_train.map({0: 5, 1: 2}) 

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=80,
    learning_rate=0.05,
    eval_metric='logloss',
    colsample_bytree=0.6,
    random_state=42
)

xgb_model.fit(X_train_combined, y_train, sample_weight=sample_weights)

# Prediccion
y_pred = xgb_model.predict(X_test_combined)

# Evaluar el modelo con Macro Average F1-Score
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.4f}")


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.40      0.48      0.44       923
           1       0.70      0.63      0.66      1762

    accuracy                           0.58      2685
   macro avg       0.55      0.55      0.55      2685
weighted avg       0.60      0.58      0.59      2685

Macro F1-Score: 0.5501


Nos guardamos el vectorizador tfidf y el modelo entrenado para facilitar el reentrenamiento y la predicción del conjunto de prueba.

In [15]:
import joblib

joblib.dump(X_train_encoded.columns, 'train_cat_columns.joblib')

# Guardamos el vectorizador TF-IDF
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

# Guardamos ek modelo entrenado
joblib.dump(xgb_model, 'xgb_model.joblib')


['xgb_model.joblib']

## 1.3 Ensemble

**Descripción del Ensemble: Voting Classifier**

Finalmente, para terminar con los árboles de decisón, se ha llevado a cabo el entrenamiento de un ensemble que combina tanto el modelo Random Forest, como el modelo XGBoost, en búsqueda de que se mejore el entrenamiento, la precisión y la robustez del model; y por tanto las métricas. 

Para ello, vamos a emplear **Voting Classifier**. Esta técnica permite elegir entre un hard voting (donde cada modelo vota por una clase y se toma la mayoría) o un soft voting (donde se promedian las prbabilidades de cada clase y se selecciona aquella con mayor probabilidad final).

**Implementación con Voting Classifier**

La implementación es similar a los dos modelos anteriores entrenados con las configuraciones ya establecidas. Cabe resaltar que al final, nos hemos decantado por un *soft voting*, la cuál es menos agresiva y clasifica mejor los datos. Además, también vamos a realifzar tanto la codificación de variables categóricas como la vectorización TF-IDF.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargarmos el dataset
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las columnas relevantes para el modelo
X = data[['subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']] 
y = data['label']  

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Codificamos las variables categóricas
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Nos asegurarnos de que ambos conjuntos tengan las mismas columnas
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1)

# Vectorización del texto empleando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(data['statement'][X_train.index])  # Usar solo las filas de X_train
X_test_tfidf = tfidf_vectorizer.transform(data['statement'][X_test.index])  # Usar solo las filas de X_test

# Concatenamos las características de texto mediante TF-IDF y las características categóricas
X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_encoded.values])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_encoded.values])

# Configuraciones de los modelos
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=80,
    learning_rate=0.05,
    eval_metric='logloss',
    colsample_bytree=0.6,
    random_state=42
)

rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',
    class_weight={0: 5, 1: 2},
    random_state=42
)

# Combinamos de los modelos en un ensemble
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_model), ('rf', rf_model)],
    voting='soft',  # 'soft' para promediar probabilidades
    n_jobs=-1
)

# Entrenamos el ensemble
voting_clf.fit(X_train_combined, y_train)

# Predicciones
y_pred = voting_clf.predict(X_test_combined)

# Evaluar el modelo
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.4f}")


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.51      0.35      0.41       923
           1       0.71      0.82      0.76      1762

    accuracy                           0.66      2685
   macro avg       0.61      0.58      0.59      2685
weighted avg       0.64      0.66      0.64      2685

Macro F1-Score: 0.5853


finalmente volvemos a guardarnos tanto el vectorizador TF-IDF como el modelo una vez entrenado

In [17]:
import joblib

X_train_encoded = pd.get_dummies(X_train, drop_first=True)

joblib.dump(X_train_encoded.columns, 'train_cat_columns.joblib')

# Guardamos el vectorizador y el modelo 
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(voting_clf, 'voting_classifier_model.joblib')


['voting_classifier_model.joblib']

## Conclusiones,
Finalmente, con el modelo Voting Classifier, que combinar Random Forest y XGBoost, se obtiene un rendimiento más solido, obteniendo una *accuracy* del 0.66 y un Mcro F1-Score de 0.5926, siendo el más robusto entre los tres modelos evaluados. Este modelo muestra un buen equilibrio entre precisión y recall, especialmente para la clase mayoritaria (fake news dectectadas), con un recall del 81% y una precisión del 71%. Para la clase minoritaria (fake news no detectadas), el recall es disminuye hasta el 37%, y la precisión disminuye hasta un 51%, reflejando que, aunque detecta menos, la predicción de esta clase es adecuada. En conjunto, el ensemble logra un desempeño más robusto y equilibrado que cualquiera de los modelos individuales.

El modelo XGBoost por sí solo presenta una precisión del 60% y un Macro F1-Score de 0.5670. Este modelo obtiene un recall del 68% para la clase mayoritaria y un 46% para la minoritaria, mostrando un mejor equilibrio que Random Forest, aunque con menor precisión para la clase minoritaria (43%).

Por último, Random Forest presenta una precisión general más baja (57%) y un Macro F1-Score de 0.5604. Aunque tiene un buen recall para la clase minoritaria (66%), su recall para la clase mayoritaria es más limitado (52%), indicando una menor capacidad para clasificar correctamente las noticias.

En resumen, el Voting Classifier combina las fortalezas de ambos modelos y consigue mejorar el equilibrio global en la clasificación, siendo el modelo más adecuado para este problema de clasificación de las diferentes fake news.

## GridSearchCV para random forest

In [ ]:
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score
import joblib

# Cargar el dataset de entrenamiento
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características y etiquetas
X = data['statement']  
y = data['label'] 
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Guardar el vectorizador para su uso posterior (si es necesario)
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

# Definir el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Definir los parámetros a probar para el modelo
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced'],
}

# Realizar GridSearchCV para encontrar los mejores parámetros
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo después de la búsqueda de hiperparámetros
best_rf_model = grid_search.best_estimator_

# Realizar predicciones con el mejor modelo
y_test_pred = best_rf_model.predict(X_test_tfidf)

# Evaluar el modelo con el Macro Average F1-Score
macro_f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")

# Reporte completo de métricas
print(classification_report(y_test, y_test_pred))

Macro Average F1-Score: 0.4105
              precision    recall  f1-score   support

           0       0.72      0.01      0.03       923
           1       0.66      1.00      0.79      1762

    accuracy                           0.66      2685
   macro avg       0.69      0.51      0.41      2685
weighted avg       0.68      0.66      0.53      2685



# 2. Entrega Kaggle

## Random Forest


In [ ]:
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Cargar el dataset de test
df_test = pd.read_csv('../../../../data/processed/test_preprocess_v1.csv')  # Asegúrate de usar la ruta correcta

# --- Preprocesamiento ---
# 1. Asegurarse de que el texto esté preprocesado de la misma manera que en el entrenamiento:
#    Esto incluye la vectorización TF-IDF.

# Cargar el vectorizador TF-IDF entrenado previamente
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Aplicar la transformación de texto en los datos de test
X_test_tfidf = tfidf_vectorizer.transform(df_test['statement'])

# --- Predicción ---
# Cargar el modelo Random Forest entrenado
rf_model = joblib.load('random_forest_model.joblib')

# Realizar las predicciones en los datos de test
y_test_pred = rf_model.predict(X_test_tfidf)

# --- Exportar Submission ---
# Crear el archivo CSV con las columnas requeridas: 'id' y 'label'
submission = pd.DataFrame({
    'id': df_test['id'],  # Usar el 'id' del dataset de prueba
    'label': y_test_pred  # Las predicciones del modelo
})

# Obtener la fecha actual en formato 'YYYY-MM-DD'
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Generar el nombre del archivo con la fecha actual
filename = f'randomforest_{current_date}.csv'

# Guardar el archivo con el nombre que incluye la fecha
submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")


## XGBoost

In [ ]:
import pandas as pd
import datetime
import joblib
import numpy as np

# Cargar el dataset de test
df_test = pd.read_csv('../../../../data/processed/test_preprocess_v1.csv')

# Columnas categóricas usadas en entrenamiento
cat_columns = ['subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']

# Cargar vectorizador y columnas categóricas guardadas en entrenamiento
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')
train_cat_columns = joblib.load('train_cat_columns.joblib')

# Codificar variables categóricas en test y alinear con columnas de entrenamiento
X_test_cat = pd.get_dummies(df_test[cat_columns], drop_first=True)
X_test_cat_aligned = X_test_cat.reindex(columns=train_cat_columns, fill_value=0)

# Vectorizar texto 'statement' en test
X_test_tfidf = tfidf_vectorizer.transform(df_test['statement'])

# Combinar características TF-IDF y variables categóricas
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_cat_aligned.values])

# Cargar modelo XGBoost entrenado
xgb_model = joblib.load('xgb_model.joblib')

# Predecir etiquetas
y_test_pred = xgb_model.predict(X_test_combined)

# Crear archivo de submission para Kaggle
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_test_pred
})

# Fecha actual para nombre del archivo
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f'xgboost_submission_{current_date}.csv'

submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")


## Voting Classifier

In [ ]:
import pandas as pd
import datetime
import joblib
import numpy as np

# Cargar el dataset de test
df_test = pd.read_csv('../../../../data/processed/test_preprocess_v1.csv')  # Ajusta la ruta si es necesario

# Cargar el vectorizador TF-IDF entrenado previamente
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Columnas categóricas usadas en entrenamiento (las que se usaron para get_dummies)
cat_columns = ['subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']

# Preprocesar las variables categóricas de test con get_dummies
X_test_cat = pd.get_dummies(df_test[cat_columns], drop_first=True)

# Vectorizar la columna de texto 'statement'
X_test_tfidf = tfidf_vectorizer.transform(df_test['statement'])

# Alinear las columnas categóricas para que coincidan con las usadas en entrenamiento
X_test_cat_aligned = X_test_cat.reindex(columns=joblib.load('train_cat_columns.joblib'), fill_value=0)

# Combinar las columnas vectorizadas TF-IDF con las categóricas codificadas
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_cat_aligned.values])

# Cargar el modelo VotingClassifier entrenado
voting_clf = joblib.load('voting_classifier_model.joblib')

# Realizar las predicciones
y_test_pred = voting_clf.predict(X_test_combined)

# Crear DataFrame para submission con las columnas requeridas
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_test_pred
})

# Guardar el archivo con timestamp para evitar sobreescrituras
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f'voting_classifier_submission_{current_date}.csv'
submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")
